In [1]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [2]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle
from astropy.time import Time

In [3]:
files = glob.glob('../IPTA_DR1/*/*.par')

In [4]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []
coords = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        ra = Angle(d['RAJ'], u.hourangle).to(u.deg)
        dec = Angle(d['DECJ'], u.degree)
        coord = SkyCoord(ra=ra, dec=dec, frame='icrs',
                     pm_ra_cosdec = float(d['PMRA'])*np.cos(dec.to(u.rad))*u.mas/u.yr, 
                     pm_dec=float(d['PMDEC'])*u.mas/u.yr,
                     obstime=Time(float(d['POSEPOCH']), format='mjd'),
                     distance=1. * u.kpc) # HACK
        coords.append(coord)
        RAs.append(ra.value)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(dec.value)
        Names.append(d['PSRJ'])
        pmra.append(coord.pm_ra_cosdec.value)
        pmdec.append(coord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        lon = float(d['ELONG'])*u.deg
        lat = float(d['ELAT'])*u.deg
        coord = SkyCoord(l=lon, b=lat, frame='galactic',
                         pm_l_cosb = float(d['PMELONG'])*np.cos(lat.to(u.rad))*u.mas/u.yr, 
                         pm_b=float(d['PMELAT'])*u.mas/u.yr,
                         obstime=Time(float(d['POSEPOCH']), format='mjd'),
                         distance=1. * u.kpc) # HACKITY HACK HACK
        icrscoord = coord.transform_to(ICRS)
        coords.append(icrscoord)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append((icrscoord.pm_ra_cosdec/np.cos(icrscoord.dec.to(u.rad))).value)
        pmdec.append(icrscoord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue
        
t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
#tmin = Table([RAs, DECs], names=['ra', 'dec'])


ValueError: Velocity data is currently only supported in the coordinate frame objects, not in SkyCoord. See the velocities documentation page for more information: http://docs.astropy.org/en/stable/coordinates/velocities.html

In [ ]:
print(coords[0].apply_space_motion(new_obstime=Time(57023., format='mjd')))

In [ ]:
gaia_time = Time(57023., format='mjd')

In [ ]:
t['ra'] = [c.apply_space_motion(new_obstime=gaia_time).ra.value for c in coords]
t['dec'] = [c.apply_space_motion(new_obstime=gaia_time).dec.value for c in coords]

In [ ]:
t.write('ipta_data.csv', format='csv', overwrite=True)
#tmin.write('ipta_data_nonames.csv', format='csv', overwrite=True)

In [ ]:
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 1.0, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/337/gaia', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open('ipta_data.csv', 'r')})

In [ ]:
print(r.text)

In [ ]:
h = open('ipta_crossmatch_1arcsecond.csv', 'w')
h.write(r.text)
h.close()